In [1]:
import numpy as np
import pandas as pd
import os
import re
import sys
from io import StringIO
import itertools
from unicodedata import normalize
import datetime
from openpyxl import load_workbook

In [6]:
src_dir = os.path.join(os.getcwd(),'SAP Input files')
input_file_dir = os.path.join(src_dir,'Original')
#clean_file_dir = os.path.join(src_dir,'Formatted')
if not os.path.exists(os.path.join(src_dir,'Formatted')):
    os.makedirs(os.path.join(src_dir,'Formatted'))
clean_file_dir = os.path.join(src_dir,'Formatted')
# usr02_columns=['Unnamed: 0', ' ', 'MANDT', 'BNAME       ', 'BCODE           ',
#        'GLTGV     ', 'GLTGB     ', 'USTYP', 'CLASS         ', 'LOCNT', 'UFLAG',
#        'ACCNT       ', 'ANAME       ', 'ERDAT     ', 'TRDAT     ', 'LTIME   ',
#        'OCOD1           ', 'BCDA1     ', 'CODV1', 'OCOD2           ',
#        'BCDA2     ', 'CODV2', 'OCOD3           ', 'BCDA3     ', 'CODV3',
#        'OCOD4           ', 'BCDA4     ', 'CODV4', 'OCOD5           ',
#        'BCDA5     ', 'CODV5', 'VERSN', 'CODVN', 'TZONE   ', 'ZBVMASTER',
#        'PASSCODE                                ', 'PWDCHGDATE', 'PWDSTATE',
#        'RESERVED', 'PWDHISTORY', 'PWDLGNDATE', 'PWDSETDATE', 'PWDINITIAL',
#        'PWDLOCKDATE',
#        'PWDSALTEDHASH                                                                                                                                                                                                                                                  ',
#        'SECURITY_POLICY                           ', 'Unnamed: 46']
# agr_1251_columns=['Unnamed: 0', ' ', 'MANDT', 'AGR_NAME                        ',
#        'COUNTER', 'OBJECT    ', 'AUTH        ', 'VARIANT', 'FIELD     ',
#        'LOW                                     ',
#        'HIGH                                    ', 'MODIFIED', 'DELETED',
#        'COPIED', 'NEU', 'NODE  ', 'Unnamed: 16']

# agr_users_columns = ['Unnamed: 0', ' ', 'MANDT', 'AGR_NAME                        ',
#        'UNAME         ', 'FROM_DAT  ', 'TO_DAT    ', 'EXCLUDE', 'CHANGE_DAT',
#        'CHANGE_TIM', 'CHANGE_TST         ', 'ORG_FLAG', 'COL_FLAG',
#        'Unnamed: 13']
# agr_agrs_columns = ['Unnamed: 0', ' ', 'MANDT', 'AGR_NAME                        ',
#        'CHILD_AGR                       ', 'ATTRIBUTES', 'Unnamed: 6']
# usr02 = pd.DataFrame(columns=usr02_columns)
# agr_1251 = pd.DataFrame(columns=agr_1251_columns)
# agr_users = pd.DataFrame(columns=agr_users_columns)
# agr_agrs = pd.DataFrame(columns=agr_agrs_columns)

In [7]:
# usr02_columns=['Unnamed:0','','MANDT','BNAME','BCODE',
# 'GLTGV','GLTGB','USTYP','CLASS','LOCNT','UFLAG',
# 'ACCNT','ANAME','ERDAT','TRDAT','LTIME',
# 'OCOD1','BCDA1','CODV1','OCOD2',
# 'BCDA2','CODV2','OCOD3','BCDA3','CODV3',
# 'OCOD4','BCDA4','CODV4','OCOD5',
# 'BCDA5','CODV5','VERSN','CODVN','TZONE','ZBVMASTER',
# 'PASSCODE','PWDCHGDATE','PWDSTATE',
# 'RESERVED','PWDHISTORY','PWDLGNDATE','PWDSETDATE','PWDINITIAL',
# 'PWDLOCKDATE',
# 'PWDSALTEDHASH',
# 'SECURITY_POLICY','Unnamed:46']

# agr_1251_columns=['Unnamed:0','','MANDT','AGR_NAME',
# 'COUNTER','OBJECT','AUTH','VARIANT','FIELD',
# 'LOW',
# 'HIGH','MODIFIED','DELETED',
# 'COPIED','NEU','NODE','Unnamed:16']

# agr_users_columns=['Unnamed:0','','MANDT','AGR_NAME',
# 'UNAME','FROM_DAT','TO_DAT','EXCLUDE','CHANGE_DAT',
# 'CHANGE_TIM','CHANGE_TST','ORG_FLAG','COL_FLAG',
# 'Unnamed:13']
# agr_agrs_columns=['Unnamed:0','','MANDT','AGR_NAME',
# 'CHILD_AGR','ATTRIBUTES','Unnamed:6']
# usr02 = pd.DataFrame(columns=usr02_columns)
# agr_1251 = pd.DataFrame(columns=agr_1251_columns)
# agr_users = pd.DataFrame(columns=agr_users_columns)
# agr_agrs = pd.DataFrame(columns=agr_agrs_columns)

In [8]:
usr02 = pd.DataFrame()
agr_1251 = pd.DataFrame()
agr_users = pd.DataFrame()
agr_agrs = pd.DataFrame()

In [9]:
for i in os.listdir(input_file_dir):
    a_file = open(os.path.join(input_file_dir,i), 'r')
    lines = a_file.readlines()
    a_file.close()
#     lines = lines[3:-1]
    del lines[0]
    del lines[1]
    del lines[-1]
    
    new_file = open(os.path.join(clean_file_dir,i), 'w')
    for line in lines:
        new_file.write(line)
    new_file.close()

In [10]:
dfs = [agr_1251, agr_agrs, agr_users, usr02]
for i,df in zip(os.listdir(clean_file_dir),dfs):
    old_stderr = sys.stderr
    result = StringIO()
    sys.stderr = result
    data = pd.read_csv(os.path.join(clean_file_dir,i), sep='|', error_bad_lines=False, encoding='latin-1')
    df = pd.concat([df,data], ignore_index=True)
    sys.stderr = old_stderr
    result_string = result.getvalue()
    with open(os.path.join(i+'_bad_lines.txt'), 'w') as bad_lines:
        for line in result_string.split(r'\n'):
            bad_lines.write(line)
            bad_lines.write('\n')
    print('OK')

OK
OK
OK
OK


In [11]:
usr02 = usr02.append(pd.read_csv(os.path.join(clean_file_dir,'USR02.txt'), dtype='str', sep='|', error_bad_lines=False, encoding='latin-1'))
agr_1251 = agr_1251.append(pd.read_csv(os.path.join(clean_file_dir,'AGR_1251.txt'), dtype='str', sep='|', error_bad_lines=False, encoding='latin-1'))
agr_users = agr_users.append(pd.read_csv(os.path.join(clean_file_dir,'AGR_USERS.txt'), dtype='str', sep='|', error_bad_lines=False, encoding='latin-1'))
agr_agrs = agr_agrs.append(pd.read_csv(os.path.join(clean_file_dir,'AGR_AGRS.txt'), dtype='str', sep='|', error_bad_lines=False, encoding='latin-1'))

b'Skipping line 43211: expected 16 fields, saw 17\n'
b'Skipping line 81121: expected 16 fields, saw 17\nSkipping line 81140: expected 16 fields, saw 17\nSkipping line 81268: expected 16 fields, saw 17\nSkipping line 83139: expected 16 fields, saw 17\nSkipping line 83272: expected 16 fields, saw 17\nSkipping line 84374: expected 16 fields, saw 17\n'
b'Skipping line 164259: expected 16 fields, saw 17\nSkipping line 165687: expected 16 fields, saw 17\nSkipping line 166995: expected 16 fields, saw 17\n'


In [12]:
usr02.columns = [c.strip(' ') for c in usr02.columns]
agr_1251.columns = [c.strip(' ') for c in agr_1251.columns]
agr_users.columns = [c.strip(' ') for c in agr_users.columns]
agr_agrs.columns = [c.strip(' ') for c in agr_agrs.columns]

In [13]:
usr02.head()

,Unnamed: 0,Md.,Usuario,Clave,Válido de,Fin valid.,Tipo usuario,Actualización de maestro de usuarios: Gr,Ctd.intentos acceso sist.fallidos,Bloqueo,...,Modelo usuario GCU,"Val.control clave acceso (SHA1, 160 bit)",Fecha últ.modificación clave acceso.5,Status modif.clave acceso,Información control GCU,Historial clv.acc.externo(USRPWDHISTORY),Fecha última entr.sist.clave acceso,Fe.: Clave acc.fijada de nuevo por adm.,Indicador: Clave acceso es inicial,Unnamed: 42
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,400,6000001185,0000000000000000,,2018-06-29,A,GENERICOS,0,64,...,,0000000000000000000000000000000000000000,2018-06-29,0,0,1,,2018-06-29,1,NaN
2,,"{x-issha, 1024}bf+Mlh5TNjUy0s5r1nlrxO0ztAxCDUn...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,400,AABBAS,0000000000000000,,2020-09-17,C,PORTALHR,0,64,...,,0000000000000000000000000000000000000000,2020-02-25,255,0,1,,2020-02-25,2,NaN
4,,"{x-issha, 1024}CDMYMsbGFKNRM3BOl1qgfLBdCBudoOH...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
agr_1251.head()

In [ ]:
agr_users.head()

In [ ]:
agr_agrs.head()

In [ ]:
log = open(os.path.join(os.getcwd(),'log.txt'), 'w')
log.writelines(['usr02 rows/columns: ', str(usr02.shape), '\n',
               'agr_1251 rows/columns: ',str(agr_1251.shape), '\n',
               'agr_users rows/columns: ', str(agr_users.shape)])
log.close()

In [ ]:
agr_1251 = agr_1251[['AGR_NAME','OBJECT', 'FIELD', 'LOW', 'HIGH', 'DELETED']]
agr_1251 = agr_1251.applymap(lambda x: str(x).strip()).applymap(lambda x: str(x).upper())
agr_1251 = agr_1251[agr_1251['DELETED'] != 'X'].reset_index(drop=True)
agr_users = agr_users[['AGR_NAME', 'UNAME', 'FROM_DAT', 'TO_DAT']]
agr_users = agr_users.applymap(lambda x: str(x).strip())
agr_users[['AGR_NAME', 'UNAME']] = agr_users[['AGR_NAME', 'UNAME']].applymap(lambda x: str(x).upper())
usr02 = usr02[['BNAME','GLTGB', 'USTYP','CLASS', 'UFLAG', 'ANAME', 'ERDAT', 'TRDAT']]
usr02[['BNAME','GLTGB', 'USTYP','CLASS', 'ANAME', 'ERDAT', 'TRDAT']] = usr02[['BNAME','GLTGB', 'USTYP','CLASS', 'ANAME', 'ERDAT', 'TRDAT']].applymap(lambda x: str(x).strip())
usr02[['BNAME','GLTGB', 'USTYP','CLASS', 'ANAME']] = usr02[['BNAME','GLTGB', 'USTYP','CLASS', 'ANAME']].applymap(lambda x: str(x).upper())

In [ ]:
# usr02 = usr02[usr02['GLTGB'] != '00.00.0000'].reset_index(drop=True)

In [ ]:
usr02['UFLAG'].value_counts()

In [ ]:
usr02[usr02['UFLAG'].isnull()]

In [ ]:
usr02['UFLAG'] = usr02['UFLAG'].astype(int)
usr02['GLTGB'] = usr02['GLTGB'].apply(lambda x: re.sub('[-.:]', '/', x))
usr02['GLTGB'] = pd.to_datetime(usr02['GLTGB'], format='%d/%m/%Y', errors = 'coerce').apply(lambda x: x.date())

In [ ]:
date_file = open(os.path.join(os.getcwd(),'Insert_period.txt'), 'r')
lines = date_file.readlines()
date_file.close()
start_date = lines[1].replace('\n', '')
start_date = pd.to_datetime(start_date).date()
end_date = lines[3].replace('\n', '')
end_date = pd.to_datetime(end_date).date()

In [ ]:
mask_usr02 = (((usr02['USTYP'] == 'A') | (usr02['USTYP'] == 'S')) & ((usr02['UFLAG'] == 0) | (usr02['UFLAG'] == 128)) & ((usr02['GLTGB'] >= start_date) | (usr02['GLTGB'].isnull())))

In [ ]:
usr02 = usr02[mask_usr02].reset_index(drop=True)

In [ ]:
agr_usr02 = pd.merge(usr02,agr_users , how='left', left_on='BNAME', right_on='UNAME', validate='many_to_many').drop('UNAME', axis=1)
agr_usr02 = agr_usr02.drop_duplicates().reset_index(drop=True)

In [ ]:
rules_grc = pd.read_excel('SA_rules.xlsx')

In [ ]:
rules_grc.columns = [re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", r), 0, re.I) for r in rules_grc.columns]
rules_grc.columns = [r.strip(' ').upper() for r in rules_grc.columns]

In [ ]:
rules_grc.columns

In [ ]:
scope_tcodes = pd.read_excel('SAP Input Files/DTT_relevant_tcodes.xlsx')
scope_tcodes['Technical_Name'] = scope_tcodes['Technical_Name'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
scope_tcodes_list = list(scope_tcodes['Technical_Name'])

In [ ]:
relevant_rules_grc = pd.DataFrame(columns = rules_grc.columns)
for tcode in scope_tcodes_list:
    mask = rules_grc[rules_grc['ACCION'] == tcode]
    relevant_rules_grc = pd.concat([relevant_rules_grc,mask])

In [ ]:
in_grc = list(relevant_rules_grc['ACCION'].unique())
out_grc = list(set(scope_tcodes_list) - set(in_grc))

In [ ]:
mask_high_value = ((agr_1251['HIGH'] != '') & (agr_1251['LOW'] != agr_1251['HIGH']) & ((agr_1251['OBJECT'] == 'S_TCODE') | \
                                            (agr_1251['OBJECT'] == 'F_BKPF_KOA') | (agr_1251['OBJECT'] == 'F_BKPF_BUK') | \
                                            (agr_1251['OBJECT'] == 'S_PROGRAM') | (agr_1251['OBJECT'] == 'F_KKKO_BUK') | \
                                            (agr_1251['OBJECT'] == 'F_ACE_PST')))

In [ ]:
agr_1251[mask_high_value].to_csv('AGR_1251_HIGHVALUES.csv')

In [ ]:
mask_low_value = ((agr_1251['HIGH'] == '') & (agr_1251['LOW'] != agr_1251['HIGH']) & ((agr_1251['OBJECT'] == 'S_TCODE') | \
                                            (agr_1251['OBJECT'] == 'F_BKPF_KOA') | (agr_1251['OBJECT'] == 'F_BKPF_BUK') | \
                                            (agr_1251['OBJECT'] == 'S_PROGRAM') | (agr_1251['OBJECT'] == 'F_KKKO_BUK') | \
                                            (agr_1251['OBJECT'] == 'F_ACE_PST')))

In [ ]:
agr_1251 = agr_1251[mask_low_value]

In [ ]:
agr_tcodes = agr_1251['LOW'][agr_1251['FIELD'] == 'TCD'].unique()

In [ ]:
tcodes_not_found = np.setdiff1d(out_grc,agr_tcodes).tolist()
out_grc = list(set(out_grc) - set(tcodes_not_found))
out_grc.sort()
tcodes_not_found = pd.Series(tcodes_not_found)

In [ ]:
tcodes_not_found.to_csv('TCODES_not_assigned.csv', header=False)

In [ ]:
object_values = ['01','02','06','10','77','S','SUBMIT','F_001','F_002']

In [ ]:
low = out_grc + object_values

In [ ]:
agr_1251_relevant = pd.DataFrame(columns=agr_1251.columns)
for l in low:
    mask = agr_1251[agr_1251['LOW'] == l]
    agr_1251_relevant = pd.concat([agr_1251_relevant,mask],ignore_index=True)

In [ ]:
agr_1251_relevant['OBJECT_LOW'] = agr_1251_relevant['OBJECT'] + "_" + agr_1251_relevant['FIELD'] + "_" + agr_1251_relevant['LOW']

In [ ]:
agr_1251_relevant = agr_1251_relevant.set_index('AGR_NAME')['OBJECT_LOW'].str.get_dummies().max(level=0).reset_index()

In [ ]:
agr_usr02 = pd.merge(agr_usr02[['BNAME','AGR_NAME']], agr_1251_relevant, how='left', on='AGR_NAME', validate='many_to_many').drop_duplicates().reset_index(drop=True)

In [ ]:
agr_usr02 = agr_usr02.loc[:,agr_usr02.columns!='AGR_NAME']
agr_usr02 = agr_usr02.groupby('BNAME', as_index=False).max().reset_index(drop=True)

In [ ]:
F_01 = (agr_usr02['S_TCODE_TCD_F-01'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_01'] == 1)
F_34 = (agr_usr02['S_TCODE_TCD_F-34'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_01'] == 1)
F19 = (agr_usr02['S_TCODE_TCD_F.19'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1))
F56 = (agr_usr02['S_TCODE_TCD_F.56'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1) & (agr_usr02['S_PROGRAM_P_GROUP_F_002'] == 1) & (agr_usr02['S_PROGRAM_P_ACTION_SUBMIT'] == 1)
F57 = (agr_usr02['S_TCODE_TCD_F.57'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1)
F81 = (agr_usr02['S_TCODE_TCD_F.81'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1)) & (agr_usr02['S_PROGRAM_P_GROUP_F_001'] == 1) & (agr_usr02['S_PROGRAM_P_ACTION_SUBMIT'] == 1)
FB11 = (agr_usr02['S_TCODE_TCD_FB11'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_KOART_S'] == 1)
FBD2 = (agr_usr02['S_TCODE_TCD_FBD2'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_02'] == 1) & (agr_usr02['F_BKPF_KOA_KOART_S'] == 1)
FBU2 = (agr_usr02['S_TCODE_TCD_FBU2'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1)
FBU8 = (agr_usr02['S_TCODE_TCD_FBU8'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_01'] == 1) & (agr_usr02['F_BKPF_KOA_KOART_S'] == 1)
FBV0 = (agr_usr02['S_TCODE_TCD_FBV0'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_06'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_77'] == 1)) & (agr_usr02['F_BKPF_KOA_ACTVT_01'] == 1)
FBV1 = (agr_usr02['S_TCODE_TCD_FBV1'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_77'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_77'] == 1)
FBV2 = (agr_usr02['S_TCODE_TCD_FBV2'] == 1) & (agr_usr02['F_BKPF_BUK_ACTVT_77'] == 1) & (agr_usr02['F_BKPF_KOA_ACTVT_77'] == 1)
FPE1 = (agr_usr02['S_TCODE_TCD_FPE1'] == 1) & (agr_usr02['F_KKKO_BUK_ACTVT_01'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1))
FPRECL = (agr_usr02['S_TCODE_TCD_FPRECL'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1))
FPYE1 = (agr_usr02['S_TCODE_TCD_FPYE1'] == 1) & (agr_usr02['F_KKKO_BUK_ACTVT_01'] == 1) & ((agr_usr02['F_BKPF_BUK_ACTVT_01'] == 1) | (agr_usr02['F_BKPF_BUK_ACTVT_02'] == 1))

In [ ]:
masks = [F_01, F_34, F19, F56, F57, F81, FB11, FBD2, FBU2, FBU8, FBV0, FBV1, FBV2, FPE1, FPRECL, FPYE1]

In [ ]:
final_df = pd.DataFrame(columns = list(agr_usr02.columns) + list(['TCODE','IN_GRC']))
for mask,tcode in zip(masks,out_grc):
    df = agr_usr02[mask]
    final_df = pd.concat([final_df,df], sort=False).reset_index(drop=True)
    final_df['TCODE'] = final_df['TCODE'].fillna(tcode)
    final_df['IN_GRC'] = final_df['IN_GRC'].fillna('NO')

In [ ]:
# ['SYSTEM', 'RULE SET', 'ACCESS RISK ID', 'ACCESS RISK DESCRIPTION',
#        'ACCESS RISK TYPE', 'ACCESS RISK LEVEL', 'BUSINESS PROCESS DESCRIPTION',
#        'FUNCTIONS', 'FUNCTION DESCRIPTION', 'ACTION', 'ACCESS RULE ID',
#        'PERMISSION OBJECT', 'FIELD', 'STATUS', 'VALUE FROM', 'VALUE TO',
#        'OWNER', 'CONDITION']

In [ ]:
relevant_rules_grc = relevant_rules_grc[['ID DE RIESGO DE ACCESO','DESCRIPCION DE RIESGO DE ACCESO','ACCION']].drop_duplicates().reset_index(drop=True)

In [ ]:
# wb = load_workbook('SA_users.xlsx')
# ws = wb['SA_users']
# data = ws.values
# columns = next(data)[0:]
# sa_users = pd.DataFrame(data, columns=columns)
# sa_users.columns = [re.sub(
#         r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
#         normalize( "NFD", r), 0, re.I) for r in sa_users.columns]
# sa_users.columns = [r.strip(' ').upper() for r in sa_users.columns]

In [ ]:
sa_users = pd.read_excel('SAP Input Files/SA_users.xlsx')
sa_users.columns = [re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", r), 0, re.I) for r in sa_users.columns]
sa_users.columns = [r.strip(' ').upper() for r in sa_users.columns]

In [ ]:
sa_users = pd.merge(sa_users[['ID USU.', 'ID DE RIESGO DE ACCESO']], relevant_rules_grc, how='left', on='ID DE RIESGO DE ACCESO', validate='many_to_many').drop_duplicates().reset_index(drop=True)
sa_users = sa_users[sa_users['ACCION'].notnull()].reset_index(drop=True)

In [ ]:
sa_users.head()

In [ ]:
sa_users.rename(columns={'ID USU.':'BNAME',
                        'ACCION':'TCODE',
                        'DESCRIPCION DE RIESGO DE ACCESO':'IN_GRC'}, inplace=True)

In [ ]:
final_df = final_df[['BNAME','TCODE', 'IN_GRC']].drop_duplicates().reset_index(drop=True)

In [ ]:
sa_users.head()

In [ ]:
del agr_1251
del usr02
del agr_agrs
# del wb

In [ ]:
final_df = pd.concat([final_df,sa_users], axis=0, ignore_index=True, sort=False)

In [ ]:
del sa_users

In [ ]:
final_df = pd.merge(final_df, scope_tcodes, how='left', left_on='TCODE', right_on='Technical_Name').drop('Technical_Name', axis=1).sort_values(by=['TCODE']).reset_index(drop=True)

In [ ]:
final_df['ID DE RIESGO DE ACCESO'][final_df['ID DE RIESGO DE ACCESO'].isnull()] = "N/A"

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('MOC_permisos_sin_nomina.csv')

In [ ]:
nomina = pd.read_excel('SAP Input Files/NOMINA.xlsx')
nomina['Nombre de usuario'] =  nomina['Nombre de usuario'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
# nomina_externos = pd.read_excel('nomina_externos.xlsx')
# nomina_externos['NOMBRE_USUARIO'] = nomina_externos['NOMBRE_USUARIO'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
# nomina_externos.rename(columns={'DEPARTAMENTO':'Departamento_ext'}, inplace=True)

In [ ]:
final_df = final_df.merge(nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='BNAME', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
# final_df = final_df.merge(nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='BNAME', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)

In [ ]:
# values = {'DESCRIPCION DE RIESGO DE ACCESO':'N/A', 'Departamento':'N/A', 'Departamento_ext':'N/A'}
# final_df = final_df.fillna(value=values)

In [ ]:
del nomina
# del nomina_externos

In [ ]:
final_df.to_csv('MOC_permisos.csv')